In [1]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import init
from torch.optim import lr_scheduler
import functools
import numpy as np
import cv2 as cv
from PIL import Image

In [2]:
def make_dataset(directory):
    
    images = []
    assert os.path.isdir(directory), '%s is not a valid directory' % directory
                        
    for root, _, fnames in sorted(os.walk(directory)):
        for fname in fnames:
            path = os.path.join(root, fname)
            images.append(path)
    return images

In [3]:
def __print_size_warning(ow, oh, w, h):
    
    if not hasattr(__print_size_warning, 'has_printed'):
        
        print("The image size needs to be a multiple of 4. "
              "The loaded image size was (%d, %d), so it was adjusted to "
              "(%d, %d). This adjustment will be done to all images "
              "whose sizes are not multiples of 4" % (ow, oh, w, h))
        
        __print_size_warning.has_printed = True

In [4]:
def __make_power_2(img, base, method=Image.BICUBIC):
    
    ow, oh = img.size
    h = int(round(oh / base) * base)
    w = int(round(ow / base) * base)
    
    if h == oh and w == ow:
        return img

    __print_size_warning(ow, oh, w, h)
    
    return img.resize((w, h), method)

In [5]:
def get_transform(method):
    
    transform_list = []
    transform_list.append(transforms.Lambda(lambda img: __make_power_2(img, base=4, method=method)))

    transform_list += [transforms.ToTensor()]
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
            
    return transforms.Compose(transform_list)

In [6]:
def init_weights(net, init_type='normal', init_gain=0.02):
    
    def init_func(m): 
        
        classname = m.__class__.__name__
        
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
           
            init.normal_(m.weight.data, 0.0, init_gain)
                
            if hasattr(m, 'bias') and m.bias is not None:
                
                init.constant_(m.bias.data, 0.0)
                
        elif classname.find('BatchNorm2d') != -1: 
            
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    net.apply(init_func)  

In [7]:
def init_net(net, init_type='normal', init_gain=0.02):
  
#     if len(gpu_ids[0]) != -1:
#         assert(torch.cuda.is_available())
#         net.to(gpu_ids[0])
#         net = torch.nn.DataParallel(net, gpu_ids)  # multi-GPUs

    init_weights(net, init_type, init_gain=init_gain)
    
    return net

In [8]:
class AlignedDataset(torch.utils.data.Dataset):

    def __init__(self, dataroot, phase):
       
        self.dataroot = dataroot
        self.phase = phase
        self.dir_A = os.path.join(dataroot, phase)  # get the image directory
        self.A_path = sorted(make_dataset(self.dir_A))  # get image paths
                               
    def __getitem__(self, index):
        
        # read a image given a random integer index
        A_path = self.A_path[index]
        A = Image.open(A_path).convert('RGB')
                               
        # apply the same transform to both A and B
        A_transform = get_transform(Image.BICUBIC)

        A = A_transform(A)

        return {'A': A, 'A_path': A_path}

    def __len__(self):
       
        return len(self.A_path)

In [9]:
class ResnetBlock(nn.Module):

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):

        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
       
        conv_block = []
        p = 0
        
        conv_block += [nn.ReflectionPad2d(1)]

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
       
        conv_block += [nn.ReflectionPad2d(1)]

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        return nn.Sequential(*conv_block)
    
    def forward(self, x):

        out = x + self.conv_block(x)  # add skip connections
        
        return out

class ResnetGenerator(nn.Module):

    def __init__(self, input_nc=3, output_nc=3, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=9, padding_type='reflect'):

        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        
        use_bias=False

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):  # add downsampling layers
            mult = 2 ** i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2 ** n_downsampling
        for i in range(n_blocks):       # add ResNet blocks
            block = ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)
            model += [block]

        for i in range(n_downsampling):  # add upsampling layers
            mult = 2 ** (n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=use_bias),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):

        return self.model(input)

In [10]:
class TestModel:

    def __init__(self):
        
#         self.gpu_ids = gpu_ids
#         self.device = torch.device('cpu')
        
        self.netG = ResnetGenerator(input_nc=3, output_nc=3, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=9, padding_type='reflect')
#         init_net(self.netG, 'normal', 0.02)
        
    def set_input(self, input):
        
#         self.input = input
        
        self.real = input['A'].to(torch.device('cpu'))
#         self.real = torch.unsqueeze(self.real, dim=0)

        self.image_paths = input['A_path']

    def forward(self):
  
        self.fake = self.netG(self.real)
        
        return self.fake

In [11]:
# Defining dataset object
dataset = AlignedDataset('D:/Data Science Projects/CNIC OCR/GANs/pytorch-CycleGAN-and-pix2pix-master/datasets', 'test_run')

# Defining the Dataloader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [12]:
data_loader

In [13]:
# Initialization of Model

# gpu_ids = [-1]
pix2pix = TestModel()

In [21]:
modelpath = 'latest_net_G.pth'
# model_details = ResnetGenerator()
model_details = torch.load(modelpath, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model = model_details['model']
# model.load_state_dict(model_details['state_dict_model'])

In [22]:
model_details.keys()

odict_keys(['model.1.weight', 'model.2.weight', 'model.2.bias', 'model.2.running_mean', 'model.2.running_var', 'model.2.num_batches_tracked', 'model.4.weight', 'model.5.weight', 'model.5.bias', 'model.5.running_mean', 'model.5.running_var', 'model.5.num_batches_tracked', 'model.7.weight', 'model.8.weight', 'model.8.bias', 'model.8.running_mean', 'model.8.running_var', 'model.8.num_batches_tracked', 'model.10.conv_block.1.weight', 'model.10.conv_block.2.weight', 'model.10.conv_block.2.bias', 'model.10.conv_block.2.running_mean', 'model.10.conv_block.2.running_var', 'model.10.conv_block.2.num_batches_tracked', 'model.10.conv_block.5.weight', 'model.10.conv_block.6.weight', 'model.10.conv_block.6.bias', 'model.10.conv_block.6.running_mean', 'model.10.conv_block.6.running_var', 'model.10.conv_block.6.num_batches_tracked', 'model.11.conv_block.1.weight', 'model.11.conv_block.2.weight', 'model.11.conv_block.2.bias', 'model.11.conv_block.2.running_mean', 'model.11.conv_block.2.running_var', '

In [23]:
model_details

OrderedDict([('model.1.weight',
              tensor([[[[-5.1934e-02, -4.8384e-02, -5.2406e-02,  ..., -2.3041e-02,
                         -7.2018e-02, -4.9572e-02],
                        [-2.4679e-02, -8.6571e-03, -2.1460e-02,  ..., -1.9343e-02,
                          1.9802e-03, -1.6116e-02],
                        [-2.2560e-02, -2.2084e-02, -1.6520e-02,  ...,  2.5181e-02,
                         -2.5513e-02, -2.3588e-02],
                        ...,
                        [-3.1166e-02,  1.6157e-03,  4.7219e-03,  ..., -3.2819e-03,
                         -1.7443e-02, -2.4623e-03],
                        [-1.3229e-02,  1.8186e-02,  4.7114e-03,  ..., -1.5149e-02,
                         -3.2355e-02, -9.2829e-03],
                        [-4.6590e-02, -1.7407e-02,  2.1337e-02,  ..., -6.4253e-02,
                         -1.9721e-02, -3.0618e-02]],
              
                       [[-4.1265e-02, -4.3557e-02, -3.7529e-03,  ...,  4.6178e-03,
                         -3.37

In [24]:
# pix2pix.netG.load_state_dict(model_details['state_dict_model'])

In [25]:
pix2pix.netG.eval()

ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), bias=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): Dropout

In [74]:
for i, data in enumerate(data_loader):
    
        pix2pix.set_input(data) 
        
        with torch.no_grad():
            pred = pix2pix.forward()      

In [75]:
image_tensor = pred.data

In [76]:
image_tensor.shape

torch.Size([1, 3, 616, 764])

In [77]:
image_numpy = image_tensor[0].cpu().float().numpy()

In [78]:
image_numpy = np.tile(image_numpy, (3, 1, 1))

In [79]:
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0

In [80]:
image_numpy = image_numpy.astype(np.uint8)

In [87]:
# image_pil = Image.fromarray(image_numpy)

In [82]:
image_numpy.save('result.png')